CREATE A REACT AGENT IN TOOLS 

In [48]:
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate

In [49]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [50]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [51]:
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [52]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [53]:
llm=ChatGroq(model="gemma2-9b-it")

In [54]:
llm.invoke("what is gen ai")

AIMessage(content='"Gen AI" is short for **Generative Artificial Intelligence**. \n\nIt\'s a type of AI that focuses on creating new content, rather than simply analyzing existing data. Think of it as AI that can imagine and build, instead of just understanding.\n\nHere\'s a breakdown:\n\n**What Gen AI can do:**\n\n* **Generate text:** Write stories, poems, articles, emails, code, and more.\n* **Create images:** Produce realistic photos, artwork, illustrations, and designs.\n* **Compose music:** Write original melodies, harmonies, and even entire songs.\n* **Generate videos:** Create short clips, animations, and even realistic deepfakes.\n* **Translate languages:** Convert text from one language to another.\n* **Summarize text:** Condense large amounts of information into concise summaries.\n\n**How Gen AI works:**\n\nGen AI models are typically trained on massive datasets of existing content. They learn the patterns and structures within this data, allowing them to generate new conten

In [55]:
loader=PyPDFLoader(file_path="C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf")
docs=loader.load()

In [56]:
documents=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=40).split_documents(docs)

In [57]:
documents

[Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='From Wikipedia, the free encyclopedia \nAn illustration of main components of the \ntransformer model from the paper \n"Attention Is All You Need"[1] is a 2017 landmark[2][3] research paper in machine learning authored by \neight scientists working at Google. The paper introduced a new deep learning architecture known as'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al.[4] It is \nconsidered a foundational[5] paper in modern artificial intelligence, as the transformer approach has \nbecome the main architecture of large language models like those based on GPT.[6][7] At the time, the'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='focus of the research w

In [58]:
db=FAISS.from_documents(documents=documents,embedding=embedding)

In [59]:
vector_db=db.as_retriever()

In [60]:
input="what is transformer"

In [61]:
vector_db.get_relevant_documents(input)

[Document(id='8178fa4c-e4b0-4f4c-a3f4-48598beb7fde', metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='"Transformer" was picked because Uszkoreit liked the sound of that word.[9] \nAn early design document was titled "Transformers: Iterative Self-Attention and Processing for \nVarious Tasks", and included an illustration of six characters from the Transformers animated show. \nThe team was named Team Transformer.[8]'),
 Document(id='606f5696-cf0d-4610-817d-de29572eb59e', metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='Some early examples that the team tried their Transformer architecture on included English-to-\nGerman translation, generating Wikipedia articles on "The Transformer", and parsing. These \nconvinced the team that the Transformer is a general purpose language model, and not just good for \ntranslation.[9] \nAs of 2024, the paper has been cited more than 100,000 t

In [62]:
from langchain.tools.retriever import create_retriever_tool

In [63]:
rag_tool=create_retriever_tool(
                                vector_db,
                                "transformer architecture",
                                "it give some information about the transformer architecture"
                                )

In [64]:
rag_tool.name

'transformer architecture'

In [65]:
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

In [66]:
search=TavilySearchResults()

In [67]:
tools=[rag_tool,search]

In [68]:
from langchain.agents import AgentExecutor,create_react_agent,self_ask_with_search,AgentType

In [69]:
from langchain import hub

In [70]:
prompt=hub.pull("hwchase17/react")

In [71]:
print(prompt.template,end=" ")

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad} 

In [72]:
#it works as
#first question will come and thought and action and then observation if it is not satisfied it
# act as a loop once the answer is satisfied then it will give the final  

In [73]:
agent=create_react_agent(llm=llm,tools=tools,prompt=prompt)

In [74]:
chain=AgentExecutor(agent=agent,tools=tools,return_intermediate_steps=True,verbose=True,handle_parsing_errors=True)

In [75]:
response1=chain.invoke({"input":"what is transformer"})



> Entering new AgentExecutor chain...
Thought: I need to know more about transformer architecture. 
Action: transformer architecture
Action Input: popular,[36] triggering a boom around large language models.[37][38] 
Since 2020, Transformers have been applied in modalities beyond text, including the vision 
transformer,[39] speech recognition,[40] robotics,[41] and multimodal.[42] The vision transformer, in turn, 
stimulated new developments in convolutional neural networks.[43] Image and video generators

networks.[31] 
AI boom era 
[edit]

From Wikipedia, the free encyclopedia 
An illustration of main components of the 
transformer model from the paper 
"Attention Is All You Need"[1] is a 2017 landmark[2][3] research paper in machine learning authored by 
eight scientists working at Google. The paper introduced a new deep learning architecture known as

accelerated on GPUs. In 2016, decomposable attention applied a self-attention mechanism 
to feedforward networks, which are easy t

In [76]:
response2=chain.invoke({"input":"who is president of india"})



> Entering new AgentExecutor chain...
Thought: I need to search for this information.
Action: tavily_search_results_json
Action Input: who is president of india[{'url': 'https://en.wikipedia.org/wiki/List_of_presidents_of_India', 'content': 'Although vested with these powers by the Constitution of India, the position is largely a ceremonial one and executive powers are de facto exercised by the prime minister.[3]\nThe president is elected by the Electoral College composed of elected members of the parliament houses, the Lok Sabha and the Rajya Sabha, and also members of the Saasana Sabha or Vidhan Sabha, the state legislative assemblies.[2] Contents\nList of presidents of India\nThe president of India is the head of state of the Republic of India and the Supreme Commander of the Indian Armed Forces. The colors used in the table indicate the following:\n(Birth–Death)\nB. D. Jatti\nM. Hidayatullah\nR. Venkataraman\nBhairon Singh Shekhawat\nM. Venkaiah Naidu\nJagdeep Dhankhar\nStatistic

In [77]:
response2

{'input': 'who is president of india',
 'output': 'Droupadi Murmu',
 'intermediate_steps': [(AgentAction(tool='tavily_search_results_json', tool_input='who is president of india', log='Thought: I need to search for this information.\nAction: tavily_search_results_json\nAction Input: who is president of india'),
   [{'url': 'https://en.wikipedia.org/wiki/List_of_presidents_of_India',
     'content': 'Although vested with these powers by the Constitution of India, the position is largely a ceremonial one and executive powers are de facto exercised by the prime minister.[3]\nThe president is elected by the Electoral College composed of elected members of the parliament houses, the Lok Sabha and the Rajya Sabha, and also members of the Saasana Sabha or Vidhan Sabha, the state legislative assemblies.[2] Contents\nList of presidents of India\nThe president of India is the head of state of the Republic of India and the Supreme Commander of the Indian Armed Forces. The colors used in the table